In [1]:
## Extract
from neo4j import GraphDatabase

driver = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=("neo4j", "mypassword"))
driver.verify_connectivity()
session = driver.session()

nodes = session.run("MATCH (s)-[t:SENT_TO]->(r) RETURN s, r, t  LIMIT 50")

In [2]:
## Transform
df = nodes.to_df()
graph = []
sender_nodes = [{'data': {'id': s.get('address'), \
                   'label': f'{s.get('address')[:1]}...{s.get('address')[s.get('address').__len__()-3:]}'}} \
                for s in df.s]
receiver_nodes = [{'data': {'id': r.get('address'), \
                   'label': f'{r.get('address')[:1]}...{r.get('address')[r.get('address').__len__()-3:]}'}} \
                 for r in df.r]
edges = [{'data': {'source': s.get('address'), \
                   'target': r.get('address'), \
                   'label': f'{t.get('tx_hash')[:4]}...{t.get('tx_hash')[t.get('tx_hash').__len__()-4:]}', \
                   'tx_hash': t.get('tx_hash'), \
                   'value': t.get('value')}} \
         for s,r,t in zip(df.s,df.r,df.t)]
graph.extend(sender_nodes)
graph.extend(receiver_nodes)
graph.extend(edges)

In [3]:
## Plot2
from dash import Dash, html, no_update, dcc
import dash_cytoscape as cyto
from dash.dependencies import Output, Input

app = Dash(__name__)

app.layout = html.Div(style = {'padding': '20px', 'backgroundColor': 'whitesmoke', 'width': '90%'}, className='column', children=[ 
        html.P("Transaction Graph"),
        html.Div(style={'display': 'flex'}, children=[
            html.P(style={'flex': '10%'}, children=["Layout style"]),
            dcc.Dropdown(
                id='layout-selector',
                style={'flex': '70%'},
                value='cose',
                clearable=False,
                options=[
                    {'label': name.capitalize(), 'value': name}
                    for name in ['breadthfirst' ,'grid', 'random', 'circle', 'cose', 'concentric']
                ]
            ),
        ]),
        cyto.Cytoscape(
            id='cytoscape-plot',
            elements=graph,
            style={'width': '100%', 'minHeight': '300px'},
            minZoom=0.2,
            maxZoom=1,
            layout={'name': 'cose'},
            stylesheet=[
                {
                    'selector':'edge[label]',
                    'style':{
                        'label':'data(label)',
                        'curve-style': 'bezier',
                        'target-arrow-shape': 'triangle',
                    }
                },
                {
                    "selector": "node[label]",
                    "style": {
                        "label": "data(label)"
                    }
                 },
            ],
        ),
        html.Div(id='selection-div', style={'minHeight': '100px', 'width': '100%'}, children=[]),
    ]),

@app.callback(
    Output('cytoscape-plot', 'layout'),
              Input('layout-selector', 'value')
)
def update_layout(layout_value):
    return {'name': layout_value, 'animate': True}

@app.callback(
    Output('selection-div', 'children'),
    Input('cytoscape-plot','selectedNodeData'),
    Input('cytoscape-plot','selectedEdgeData')
)
def update_layout(selected_nodes, selected_edges):
    if selected_nodes is None and selected_edges is None:
        return no_update
    else:
        output = []
        if selected_nodes is not None:
            output.extend([html.Li(f"Node => Address: {x['id']}") for x in selected_nodes])
        if selected_edges is not None:
            output.extend([html.Li(f"Edge => Sender: {x['source']}, \
                Receiver: {x['target']}, \
                TxHash: {x['tx_hash']},  \
                Value(lovelace): {x['value']}") for x in selected_edges])
        return html.Div([
                    html.H3("Selections"),
                        html.Ul(output)])    

app.run_server(debug=True, mode='jupyterlab')

In [4]:
## Plot (Basic Version)
# from dash import Dash, html
# import dash_cytoscape as cyto

# app = Dash(__name__)

# app.layout = html.Div(style = {'padding': '20px', 'backgroundColor': 'whitesmoke', 'width': '90%'}, className='column', children=[ 
#         html.P("Transaction Graph"),
#         cyto.Cytoscape(
#             id='cytoscape-plot',
#             elements=graph,
#             style={'width': '100%', 'minHeight': '300px'},
#             minZoom=0.2,
#             maxZoom=1,
#             layout={'name': 'cose', 'padding': 100},
#             stylesheet=[
#                 {
#                     'selector':'edge[label]',
#                     'style':{
#                         'label':'data(label)',
#                         'curve-style': 'bezier',
#                         'target-arrow-shape': 'triangle',
#                     }
#                 },
#                 {
#                     "selector": "node[label]",
#                     "style": {
#                         "label": "data(label)"
#                     }
#                  }
#             ],
#         ),
#     ]),   

# app.run_server(debug=True, mode='jupyterlab')